# Neural Machine Translation
This article takes you through the core concepts of the following...
- English to German translation using Neural Machine Translation(NMT)
- It uses LSTM networks with Attention
- Beyond translation MT deciphers word sense disambiguation (e.x. `bank` refers to `financial bank` or `riverside bank`
- Implemented using RNN with LSTMs can work for short to medium sentences but can result in vanishing gradient for long sequences
- To address this, an attention mechanism is used to allow the decoder to access all relevant parts of the input sentence regardless of its lenght

1. Preprocess the training and eval data
2. Implement an encoder-decoder system with attention
3. Understand how attention works
4. Build the NMT model from scratch using Trax
5. Generate translations using `Greedy and Minimum Bayes Risk`(MBR) decoding

## Part 1. Data Preparation

### 1.1 Importing the Data

In [1]:
from termcolor import colored
import random
import numpy as np

import trax
from trax import layers as tl
from trax.fastmath import numpy as fastnp
from trax.supervised import training

DATA_DIR = './data/01'

!pip list | grep trax # trax == 1.3.4 is required 

INFO:tensorflow:tokens_length=568 inputs_length=512 targets_length=114 noise_density=0.15 mean_noise_span_length=3.0 
trax                          1.3.4


In [2]:
# Get generator function for the training set
# This will download the train dataset if no data_dir is specified
train_stream_fn = trax.data.TFDS(
    'opus/medical',
    data_dir=DATA_DIR,
    keys=('en', 'de'),
    eval_holdout_size=0.01, #1% for eval
    train=True
)

# Get generator function for the eval set
eval_stream_fn = trax.data.TFDS(
    'opus/medical',
    data_dir=DATA_DIR,
    keys=('en', 'de'),
    eval_holdout_size=0.01, #1% for eval
    train=False
)

In [3]:
train_stream = train_stream_fn()
print(colored('train data (en, de) tuple:', 'red'), next(train_stream))
print()

eval_stream = eval_stream_fn()
print(colored('eval data (en, de) tuple:', 'red'), next(eval_stream))

train data (en, de) tuple: (b'Sverige BRISTOL-MYERS SQUIBB AB Tel: + 46 8 704 71 00\n', b'Sverige BRISTOL-MYERS SQUIBB AB Tel: + 46 8 704 71 00\n')

eval data (en, de) tuple: (b'Lutropin alfa Subcutaneous use.\n', b'Pulver zur Injektion Lutropin alfa Subkutane Anwendung\n')


### 1.2 Tokenization and Formatting
- Tokenizing the sentences using subword representations
- Each sentences is represented as an array of integers 
- To avoid out-of-vocab words, subword representations are used
- For example, instead of having separate entries in your vocabulary for --"fear", "fearless", "fearsome", "some", and "less"--, you can simply store --"fear", "some", and "less"-- then allow your tokenizer to combine these subwords when needed.
- This allows it to be more flexible, wont have to save uncommon words explicitly

In [4]:
# Global variables that state the filename and directory of the vocabulary file
VOCAB_FILE = 'ende_32k.subword'
VOCAB_DIR = './data/01'

In [5]:
# Tokenize the dataset.
tokenized_train_stream = trax.data.Tokenize(vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)(train_stream)
tokenized_eval_stream = trax.data.Tokenize(vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)(eval_stream)

In [6]:
# Append EOS at the end of each sentence
# Integer assigned as end of sentence (EOS)
# This will help us to infer the model has completed the translation
EOS = 1

# Generator helper function to append EOS to each sentence
def append_eos(stream):
    for (inputs, targets) in stream:
        inputs_with_eos = list(inputs) + [EOS]
        targets_with_eos = list(targets) + [EOS]
        targets_with_eos = list(targets) + [EOS]
        
        yield np.array(inputs_with_eos), np.array(targets_with_eos)
        
# Append EOS to the train data
tokenized_train_stream = append_eos(tokenized_train_stream)
tokenized_eval_stream = append_eos(tokenized_eval_stream)

In [7]:
# Filter long sentences
# Filter too long sentences to not run out of memory
# length_keys=[0, 1] means we filter both English and German sentences 
# Both must be not longer than 256 tokens for training / 512 for eval
filtered_train_stream = trax.data.FilterByLength(
    max_length=256,
    length_keys=[0, 1]
)(tokenized_train_stream)
filtered_eval_stream = trax.data.FilterByLength(
    max_length=512,
    length_keys=[0,1]
)(tokenized_eval_stream)

train_input, train_target = next(filtered_train_stream)
print(colored(f'Single tokenized example input:', 'red' ), train_input)
print(colored(f'Single tokenized example target:', 'red'), train_target)

Single tokenized example input: [14026  2801  3551 32955   135   150 14443 22008 21980   332 30650  4729
   992     1]
Single tokenized example target: [14026  2801  3551 32955   135   150 14443 22008 21980   332 30650  4729
   992     1]


### 1.3 Tokenize and Detokenize Helper Functions

In [8]:
def tokenize(input_str, vocab_file=None, vocab_dir=None):
    EOS = 1
    inputs = next(trax.data.tokenize(
        iter([input_str]),
        vocab_file=vocab_file,
        vocab_dir=vocab_dir
    ))
    inputs = list(inputs) + [EOS]
    batch_inputs = np.reshape(np.array(inputs), [1, -1])
    
    return batch_inputs

def detokenize(integers, vocab_file=None, vocab_dir=None):
    integers = list(np.squeeze(integers))
    EOS = 1
    if(EOS in integers):
        integers = integers[:integers.index(EOS)]
        
    return trax.data.detokenize(integers, vocab_file=vocab_file, vocab_dir=vocab_dir)

In [9]:
# Detokenize an input-target pair of tokenized sentences
print(colored(f'Single detokenized example input:', 'red'), detokenize(train_input, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR))
print(colored(f'Single detokenized example target:', 'red'), detokenize(train_target, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR))
print()

# Tokenize and detokenize a word that is not explicitly saved in the vocabulary file.
# See how it combines the subwords -- 'hell' and 'o'-- to form the word 'hello'.
print(colored(f"tokenize('hello'): ", 'green'), tokenize('hello', vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR))
print(colored(f"detokenize([17332, 140, 1]): ", 'green'), detokenize([17332, 140, 1], vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR))

Single detokenized example input: Tel: +421 2 57 103 777

Single detokenized example target: Tel: +421 2 57 103 777


tokenize('hello'):  [[17332   140     1]]
detokenize([17332, 140, 1]):  hello


### 1.4 Bucketing
[Comprehensive Hands-on Guide to Sequence Model batching strategy: Bucketing technique](https://medium.com/@rashmi.margani/how-to-speed-up-the-training-of-the-sequence-model-using-bucketing-techniques-9e302b0fd976)

In [10]:
# Bucketing to create streams of batches

# Buckets are defined in terms of boundaries and batch sizes
# batch_sizes[i] determines the batch size for items with length < boundaries[i]
# So below, we'll take a batch of 256 sentences of length < 8, 128, if length is
# between 8 and 16. and so on -- and only 2 if length is over 512
boundaries = [8, 16, 32, 64, 128,256, 512]
batch_sizes = [256, 128, 64, 32, 15, 8, 4, 2]

# Create the generators
train_batch_stream = trax.data.BucketByLength(
    boundaries, 
    batch_sizes,
    length_keys=[0,1]
)(filtered_train_stream)

eval_batch_stream = trax.data.BucketByLength(
    boundaries,
    batch_sizes,
    length_keys=[0,1]
)(filtered_eval_stream)

# Add masking for the padding (0s)
train_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(train_batch_stream)
eval_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(eval_batch_stream)

### 1.5 Exploring the Data

In [11]:
input_batch, target_batch, mask_batch = next(train_batch_stream)

# let's see the data type of a batch
print("input_batch data type: ", type(input_batch))
print("target_batch data type: ", type(target_batch))

# let's see the shape of this particular batch (batch length, sentence length)
print("input_batch shape: ", input_batch.shape)
print("target_batch shape: ", target_batch.shape)

input_batch data type:  <class 'numpy.ndarray'>
target_batch data type:  <class 'numpy.ndarray'>
input_batch shape:  (32, 64)
target_batch shape:  (32, 64)


- The tokens acquired are used to produce embedding vectors for each word in the sentence
- Hence, the embedding for a sentence is a matrix
- The number of sentence in each batch us usually a power of 2 for optimal computer memory usage

In [12]:
# pick a random index less than the batch size.
index = random.randrange(len(input_batch))

# use the index to grab an entry from the input and target batch
print(colored('THIS IS THE ENGLISH SENTENCE: \n', 'red'), detokenize(input_batch[index], vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR), '\n')
print(colored('THIS IS THE TOKENIZED VERSION OF THE ENGLISH SENTENCE: \n ', 'red'), input_batch[index], '\n')
print(colored('THIS IS THE GERMAN TRANSLATION: \n', 'red'), detokenize(target_batch[index], vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR), '\n')
print(colored('THIS IS THE TOKENIZED VERSION OF THE GERMAN TRANSLATION: \n', 'red'), target_batch[index], '\n')

THIS IS THE ENGLISH SENTENCE: 
 Following subcutaneous injection, onset of action is within 30 minutes, the phase of maximum action is between 1 and 4 hours after injection and the duration of action is 7 to 9 hours.
 

THIS IS THE TOKENIZED VERSION OF THE ENGLISH SENTENCE: 
  [32162  5457  8347 20808  1183 28751     2  3192   487     7   588    16
   374   448  1379     2     4  3377     7  4433   588    16   255   135
     8   219  1476   288 28751     8     4 15384     7   588    16   332
     9   444  1476  3550 30650  4729   992     1     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0] 

THIS IS THE GERMAN TRANSLATION: 
 Bei subkutaner Gabe beginnt die Wirkung innerhalb von 30 Minuten, liegt die Phase maximaler Wirkung 1 bis 4 Stunden nach Injektion und die Wirkdauer zwischen 7 und 9 Stunden.
 

THIS IS THE TOKENIZED VERSION OF THE GERMAN TRANSLATION: 
 [  752  5457  5261 22575    76 10639    35  6565    10  58

## Part 2. NMT with Attention

### 2.1 Attention Overview
- An attention model will be built using an encoder-decoder architecture
- The RNN will take in a tokenized version of a sentence in its encoder.
- Pass the tokenized data into the decoder for translation
- Using a sequence-to-sequence model with LSTMs will work effectively for short to medium sentences but will degrade for longer ones
- All the context of the input sentence is compressed into one vector and passed into a decoder block
- Context of the first parts of the input will have very little effect on the final vector passed to the decoder

$$ENCODER \rightarrow \small{hello}\hspace{2mm} \normalsize{how}\hspace{2mm} \large{are}\hspace{2mm} \Large{you}\hspace{2mm} \huge{today}\hspace{2mm} \Huge{!} \normalsize \rightarrow DECODER$$

- Adding an attention layer to this model avoids this problme by giving the decoder access to all parts of the input sentence
- In a 4 word input sentence,
    - Remember that a hidden state is produced at each timestep of the encoder
    - These hidden states are all passed to the attention layer and each are given a score given the current activation(ie hidden state) of the decoder
    - ie After predicting the first word, the attention layer will receive all the encoder hidden states as well as decoder hiddent state when producing the word wie
    - Given these information, it will score each of the encoder hidden states to know which one the decoder should focus on to produce the next word
- The result of the model training might have learned that it should align to the second encoder hidden state a
- Subsequently assigns a high probability to the word geht.
- If we use greedy decoding, we will output the said word as the next symbol, 
- then restart the process to produce the next word until we reach an end sentence prediction

This is Scaled Dot Product Attention of the form
$$Attention(Q, K, V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$$
- Computing scores using Queries(Q) and keys(K) followed by a multiplication values(V) to get a context vector at a particular timestep of the decoder
- The context vector is fed to the decoder RNN to get a set of probabilities for the next predicted word
- The division by squre root of the keys dimensionality $(\sqrt{d_k})$ is for improving model performance
- The encoder activations (hidden states) will be the keys and values, while decoder activations will be queries

#### 2.2.1 Input Encoder
- The input encoder runs on the input tokens, creates its embeddings and feeds it to an LSTM network.
- This outputs the activations that will be the keys and values for attention
- It is a serial network with `tl.Embedding` and `tl.LSTM`

In [13]:
def input_encoder_Fn(input_vocab_size, d_model, n_encoder_layers):
    input_encoder = tl.Serial(
        tl.Embedding(input_vocab_size, d_model),
        [tl.LSTM(d_model) for _ in range(n_encoder_layers)]
    )
    return input_encoder

In [14]:
import w1_unittest

w1_unittest.test_input_encoder_fn(input_encoder_Fn)

 All tests passed


#### 2.2.2 Pre-attention Decoder
- The pre-attention decoder runs on the targets and creates activations that are used as queries in attention
- This is a Serial netwokr which is composed of the `tl.ShiftRight`, `tl.Embedding`, `tl.LSTM`

In [15]:
def pre_attention_decoder_fn(mode, target_vocab_size, d_model):
    pre_attention_decoder = tl.Serial(
        # Shift right to insert start-of-sentence token and implement 
        # teacher forcing during training
        tl.ShiftRight(mode),
        tl.Embedding(target_vocab_size, d_model),
        tl.LSTM(d_model)
    )
    return pre_attention_decoder

In [16]:
w1_unittest.test_pre_attention_decoder_fn(pre_attention_decoder_fn)


 All tests passed
